# Scikit-learn: dane tekstowe

Aby przeprowadzić machine learning na dokumentach tekstowych, najpierw musimy zmienić treść tekstu na numeryczne wektory cech.

Reprezentacja bag-of-words
Przypisać stałą liczbę całkowitą id do każdego słowa występującego w dowolnym dokumencie zestawu treningowego 
(na przykład przez utworzenie słownika ze słów na indeksy). 
Dla każdego dokumentu j, zliczyć w nim liczbę wystąpień każdego słowa k, zapisując ją w X[k,j]

### Tokenizacja i zamiana tekstów na wektory

Pierwszy krok w analizie kolekcji dokumentów to tokenizacja i filtrowanie stop-wordów (ocjonalnie). Zawarte są w komponencie typu Vectorizer (np CountVectorizer), który buduje słownik mapowania słów na indeksy i przekształca dokumenty w wektory frekwencyjne bag-of-words.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(['dokument pierwszy.', 'To jest tekst drugiego dokumentu'])

Zapoznaj się z innymi sposobami wektoryzacji dostępnymi w scikit-learn:
http://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text

### Trenowanie modelu klasyfikatora i predykcja

Dysponując cechami, funkcjami, możemy zacząć trenować klasyfikator, aby spróbować przewidzieć etykietę każdego dokumentu. Przykład dla klasyfikatora naiwnego Bayesa. Scikit-learn zawiera kilka wariantów tego klasyfikatora; Najbardziej odpowiedni do liczenia słów jest wariant MultinomialNB:

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, Y_train)

Wywołanie metody fit() powoduje dopasowanie się modelu do danych treningowych oraz etykiet tych danych.
Obiekt clf jest wytrenowanym modelem klasyfikatora który możemy używać do etykietowania nowych, niewidzianych obserwacji. Robimy to wywołując na nim funkcję predict:

In [ ]:
docs_new = ['No hipsters write tweets', 'Machine learning is fun?']
X_new_counts = count_vect.transform(docs_new)
predicted = clf.predict(X_new_tfidf)

### Tworzenie pipelines (potoków?)

Czasem warto dołożyć jeszcze kolejny krok przetwarzania, pomiędzy wektoryzacją a klasyfikatorem. Na przykład transformację TF-IDF, selekcję cech lub redukcję wymiarowości.

Aby uczynić sekwencję transformacji: wektorizer => transformator => klasyfikator łatwiejszą do pracy, scikit-learn udostępnia klasę Pipeline, która zachowuje się jak klasyfikator, jest jednak złożona z większej liczby elementów:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

Na obiekcie tego typu działają metody fit i predict, typowe dla prostych klasyfikatorów.

In [ ]:
_ = text_clf.fit(X_train, Y_train)
Y_pred = text_clf.predict(X_test) # przewidywanie etykiet danych testowych (Y_pred)

### Ewaluacja modeli

Scikit-learn zapewnia narzędzia do analizy wyników:

In [ ]:
from sklearn import metrics
print(metrics.classification_report(Y_true, Y_pred))  # szczegolowa analiza, wszystkie miary
print(metrics.confusion_matrix(Y_true, Y_pred))   # macierz konfuzji / confusion matrix

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('Tweets.csv')
df = df[df['airline_sentiment'] != 'neutral']

len(df)
X = df['text']
y = df['airline_sentiment']

X_train = X[:7000]
X_validate = X[7000:9000]
X_test = X[9000:]


y_train = y[:7000]
y_validate = y[7000:9000]
y_test = y[9000:]


#podmiana na 0/1:
y = y.map({'negative': 0, 'positive': 1}).values

def print_significant_features(pipeline=None, n=20):
    feature_names = pipeline.get_params()['vect'].get_feature_names()
    coefs = []
    try:
        coefs = pipeline.get_params()['clf'].coef_
    except:
        coefs.append(pipeline.get_params()['clf'].feature_importances_)
    print("Total features: {}".format(len(coefs[0])))
    coefs_with_fns = sorted(zip(coefs[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_2, fn_2) in top:
        print( coef_2, str(fn_2) )


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
text_clf = Pipeline([('vect', TfidfVectorizer(stop_words='english')),
                      ('clf', RandomForestClassifier()),
 ])

text_clf.fit(X_train, y_train)

print_significant_features(text_clf)

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('Tweets.csv')

In [ ]:
df

In [ ]:
len(df)

In [ ]:
df = df[df['airline_sentiment'] != 'neutral']

In [ ]:
len(df)

In [ ]:
X = df['text']
y = df['airline_sentiment']

X_train = X[:7000]
X_validate = X[7000:9000]
X_test = X[9000:]


y_train = y[:7000]
y_validate = y[7000:9000]
y_test = y[9000:]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
text_clf = Pipeline([('vect', TfidfVectorizer(stop_words='english')),
                      ('clf', RandomForestClassifier()),
 ])

text_clf.fit(X_train, y_train)

In [ ]:
Y_pred = text_clf.predict(X_test)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, Y_pred))  # szczegolowa analiza, wszystkie miary
print(metrics.confusion_matrix(y_test, Y_pred))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
art = tfidf.fit_transform(articles)
art